In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last" # all | last | last_expr | none 

In [ ]:
# for name in dir():
#     if not name.startswith('_'):
#         del globals()[name]

# !tar -xf ./data/images.tar.gz
# !tar -xf ./data/annotations.tar.gz

### Data preparation and model configuration

In [ ]:
# ============= Import required packaages ==============
import time

# Import all custom variables and modules
from custom_classes_defs.preprocessing import *
from custom_classes_defs.hed import *

RND_STATE = 247
BATCH_SIZE = 64
FREEZE = 0
EPOCHS = 200
keras.utils.set_random_seed(RND_STATE)
from keras.utils import plot_model

INTERACTIVE_SESSION = False

# -------------------------------------------------------

In [ ]:
# Verify tensorflow/keras versions
print(f"tensorflow version: {tf.__version__}")
print(f"keras version: {keras.__version__}")

# Verify CPU/GPU availability
print(tf.config.list_physical_devices())
NUM_GPU = len(tf.config.list_physical_devices('GPU'))
print(f"Number of GPUs assigned for computation: {NUM_GPU}")

if NUM_GPU:
    # print GPU info
    !nvidia-smi

In [ ]:
pets = Oxford_Pets(
        input_dir = "./data/images/",
        target_dir = "./data/annotations/trimaps/",
        img_size = (160, 160),
        batch_size = BATCH_SIZE
    )

train_dataset, valid_dataset, test_dataset = \
        pets.split_data(train_ratio=0.5, val_ratio=0.2, seed=RND_STATE)


print(f"training data (size = {pets.train_size})")
print(f"validation data (size = {pets.validation_size})")
print(f"test data (size = {pets.test_size})")
print("Data images tensor:",train_dataset.element_spec[0])
print("Data labels tensor:",train_dataset.element_spec[1])


In [ ]:

# Model configurations
conf = model_config(
    epochs=100,
    loss="sparse_categorical_crossentropy",
    batch_size=BATCH_SIZE,
    save_path=f'./oxford-pets/hed{FREEZE}',
    img_shape=pets.img_size,
    target_size=pets.img_size,
    train_size=pets.train_size,
    test_size=pets.test_size,
    validation_size=pets.validation_size,
    channels_dim=(3,3),
    pos_label=pets.pos_label,
    multiple_gpu_device=(NUM_GPU>1)
)

assert not(conf.new_training_session)

out_no = 4
hed_out = f'output{out_no}_'

# conf.double_check(INTERACTIVE_SESSION)
conf.info()


### Build model

In [ ]:
print("\n\n{}\n\t{}\n{}".format('='*55,f'Build model', '-'*55))

if conf.multiple_gpu_device:

    strategy = tf.distribute.MirroredStrategy()
    print(f"Number of devices: {strategy.num_replicas_in_sync}")

m_obj = HED2D(model_arch=conf.model_arch)
model = m_obj.build_model()


# model.summary()
# keras.utils.plot_model(model, 'm_obj.png',show_shapes=True)
# plot_model(model, 'm_obj.png',show_shapes=True)
num_trainable_weights = sum([np.prod(w.shape) for w in model.trainable_weights])
print(f"Total number of parameters: {model.count_params():,}")
print(f"Total trainable wieghts: {num_trainable_weights:,}")
print(f"Total non-trainable wieghts: {model.count_params()-num_trainable_weights:,}")



### Train the model

In [ ]:
print("\n\n{}\n\t{}\n{}".format('='*55,f'Train {m_obj.Name} model', '-'*55))

train_history = None
# model, train_history = \
#     conf.execute_training(
#         model, 
#         data=None, 
#         plot_history=INTERACTIVE_SESSION
# )

best_model_track = sorted(glob(conf.save_path+'/*.h5'))
if len(best_model_track):
    model.load_weights(best_model_track[-1])
    print("Best model weights loaded!")
else:
    print(f"No model weights found in {conf.save_path}!")

In [ ]:
if INTERACTIVE_SESSION:
    show_convergence(train_history.history, [hed_out+'accuracy','val_'+hed_out+'accuracy'])

In [ ]:
if INTERACTIVE_SESSION:
    show_convergence(train_history.history, [hed_out+'f1_score','val_'+hed_out+'f1_score'])

In [ ]:
if INTERACTIVE_SESSION:
    show_convergence(train_history.history, 'lr')

### Visualize predictions

In [ ]:
print("\n\n{}\n\t{}\n{}".format('='*55,f'Evaluate {m_obj.Name} model', '-'*55))
y_preds = model.predict(test_dataset, verbose=2)


In [ ]:

if INTERACTIVE_SESSION:
    for out_no in range(6):
        pets.display_sample_image(y_preds[out_no], 'test')
    

In [ ]:
# Using scikit-learn
for out_no in range(6):
    scores = m_obj.evaluate_sklearn(test_dataset, y_preds[out_no],report=True, average='weighted')
    print(scores)